In [1]:
import os
import cv2
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
import shutil
import numpy as np
import tensorflow as tf

#print("\n\t tf:",tf.__version__)
#tf.enable_eager_execution()
# from tensorflow.keras.utils import plot_model
#from yolov3.dataset import Dataset
from yolov3.yolov4 import compute_loss1
from yolov3.utils import load_yolo_weights
from yolov3.configs import *
from evaluate_mAP import get_mAP

print("\n\t YOLO_TYPE:",YOLO_TYPE)

import os
import cv2
import random
import numpy as np
import tensorflow as tf
import pickle
from yolov3.utils import read_class_names, image_preprocess#,draw_bbox
from yolov3.yolov3 import bbox_iou
from yolov3.configs import *
from yolov3.utils import load_yolo_weights, detect_image, image_preprocess, postprocess_boxes, nms, read_class_names
from supportFunctions import draw_bbox
'''
    more details regarding Dataset1 is in yolo3PhoscModify2.ipynb
'''

from yolov3.dataset import Dataset1


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12150414845287389058
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10299730816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 59170098176290254
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1"
]

	 YOLO_TYPE: yolov3


In [2]:
#from yolov3.yolov4 import Create_YoloPhosc
from yolov3.yolov3 import Create_Yolov3
global TRAIN_FROM_CHECKPOINT

gpus = tf.config.experimental.list_physical_devices('GPU')
print(f'GPUs {gpus}')
if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

trainset = Dataset1('train')
testset = Dataset1('test')


if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

steps_per_epoch = len(trainset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
total_steps = TRAIN_EPOCHS * steps_per_epoch

if YOLO_TYPE == "yolov3":
    Darknet_weights = YOLO_V3_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V3_WEIGHTS


if TRAIN_TRANSFER:
    #Darknet = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)
    Darknet = Create_Yolov3(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)

    #print("Darknet_weights=",os.path.isfile(Darknet_weights))
    load_yolo_weights(Darknet, Darknet_weights) # use darknet weights
    
#yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)

yolo=Create_Yolov3(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)

if TRAIN_FROM_CHECKPOINT:
    try:
        print("\n\t load weights!!!")
        yolo.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}")
        print("\n\t loading complete")

    except ValueError:
        print("Shapes are incompatible, transfering Darknet weights")
        TRAIN_FROM_CHECKPOINT = False
'''
if TRAIN_TRANSFER and not TRAIN_FROM_CHECKPOINT:
    for i, l in enumerate(Darknet.layers):
        layer_weights = l.get_weights()
        if layer_weights != []:
            try:
                yolo.layers[i].set_weights(layer_weights)
            except:
                print("skipping", yolo.layers[i].name)
'''
optimizer = tf.keras.optimizers.Adam()

#mAP_model = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES) # create second model to measure mAP
#best_val_loss = 1000 # should be large at start

#print("\n\t mAP_model =",mAP_model)

#print("\n\t yolo:",yolo.summary())





GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
def validate_step(image_data, target):
    with tf.GradientTape() as tape:
        pred_result = yolo(image_data, training=False)
        
        print("\n\t 1.pred_result validate:",len(pred_result[0]))
        pred_result =pred_result[0]
        print("\n\t 2.pred_result validate:",pred_result.shape)

#         if type(pred_result) is list:
#             pred_result =pred_result[0]
        giou_loss=conf_loss=prob_loss=0

        # optimizing process
        grid = 1 if not TRAIN_YOLO_TINY else 2
        for i in range(grid):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            loss_items = compute_loss1(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]

        total_loss = giou_loss + conf_loss + prob_loss

    return giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()

phoscLoss=0
def train_step(nm,counter,image_data, target):
    
    with tf.GradientTape() as tape:
        pred_result = yolo(image_data, training=True)
        '''
        print("\n\t image_data in train:",image_data.shape)
        print("\n\t in train pred_result len=",len(pred_result))
        print("\n\t in train pred_result 0=",pred_result[0].shape)
        print("\n\t in train pred_result 1=",pred_result[1].shape)
        print("\n\t in train pred_result 0=",pred_result[2].shape)
        print("\n\t in train pred_result 1=",pred_result[3].shape)
        '''
    
        image2=tf.squeeze(image_data)
    
        giou_loss=conf_loss=prob_loss=phocLoss1=phosLoss2=0

        # optimizing process
        grid = 3 if not TRAIN_YOLO_TINY else 2

        for i in range(grid):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            #print("\n\t conv train:",conv.shape,"\t i:",i)
            #print("\n\t pred train:",pred.shape,"\t i:",i)
            
            loss_items = compute_loss1(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]
            phocLoss1+=loss_items[3]
            phosLoss2+=loss_items[4]

            
        #print("\n\t phoscLoss1:",phoscLoss1,"\t phoscLoss2:",phoscLoss2)
        total_loss = giou_loss + conf_loss + prob_loss
        #total_loss1=phoscLoss+giou_loss + conf_loss + prob_loss+phocLoss1+phosLoss2
        total_loss1=0.7*phoscLoss+0.3*total_loss
        gradients = tape.gradient(total_loss1, yolo.trainable_variables)
        optimizer.apply_gradients(zip(gradients, yolo.trainable_variables))

        global_steps.assign_add(1)
        if global_steps < warmup_steps:# and not TRAIN_TRANSFER:
            lr = global_steps / warmup_steps * TRAIN_LR_INIT
        else:
            lr = TRAIN_LR_END + 0.5 * (TRAIN_LR_INIT - TRAIN_LR_END)*(
                (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi)))
        optimizer.lr.assign(lr.numpy())

        # writing summary data
        with writer.as_default():
            tf.summary.scalar("lr", optimizer.lr, step=global_steps)
            tf.summary.scalar("total_loss", total_loss, step=global_steps)
            tf.summary.scalar("total_loss1", total_loss1, step=global_steps)

            tf.summary.scalar("giou_loss", giou_loss, step=global_steps)
            tf.summary.scalar("conf_loss", conf_loss, step=global_steps)
            tf.summary.scalar("prob_loss", prob_loss, step=global_steps)

            tf.summary.scalar("phocLoss1", phocLoss1, step=global_steps)
            tf.summary.scalar("phosLoss2",phosLoss2, step=global_steps)
            tf.summary.scalar("phocLoss1+phosLoss2",(phocLoss1+phosLoss2), step=global_steps)

        writer.flush()
    print("\n\t total_loss =",total_loss.numpy(),"\t total_loss1 =",total_loss1.numpy())
    print("\n\t phocLoss1=",phocLoss1.numpy(),"\t phosLoss2:",phosLoss2.numpy())

        
    pred_result = yolo(image_data, training=False)
    #print("\n\t image_data pred:",image_data.shape)
    #print("\n\t len(pred_result) pred:",len(pred_result))
    image2=tf.squeeze(image_data)

    #print("\n\t\t 1.pred_result pred=",pred_result[0].shape)
    #print("\n\t\t 2.pred_result pred=",pred_result[1].shape)
    #print("\n\t\t 3.pred_result pred=",pred_result[2].shape)
    
    #print("\n\t image2=",image2.shape)
    #pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_result]
    #print("\n\t pred_bbox =",pred_bbox )
    #pred_bbox = tf.concat(pred_bbox, axis=0)


    # optimizing process
    grid = 3 if not TRAIN_YOLO_TINY else 2
    for i in range(grid):

        '''
            these r 3 feature maps of size 52*52,26*26,13*13,
                                
        '''
        conv, pred = pred_result[i*2], pred_result[i*2+1]

    
#         print("\n\t\t conv pred:",conv.shape)
#         print("\n\t\t pred pred=",pred.shape)
    

        ''' 
            (xc,yc,w,h,obj,pro) is last dimension of size 6

            pred = (1, 13, 13, 3, 6) is reshaped to 
            pred_bbox = (507, 6)   169 is len of single anchor tot 3 vectors             
             #2.pred_result: (1, 52, 52, 2325)
            conv: (1, 26, 26, 18)
            pred = (1, 26, 26, 3, 6)
            pred_bbox = (2028, 6)  676 is lenght of single anchor tot 3 vectors

            image_data= (1, 416, 416, 3)
            pred = (1, 52, 52, 3, 6)
            pred_bbox = (8112, 6)  2704 is len of single anchor tot 3 vectors

        '''

        pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred]
        #print("\n\t 1.pred_bbox pred=",len(pred_bbox))

        pred_bbox = tf.concat(pred_bbox, axis=0)

        #print("\n\t 2.pred_bbox pred=",pred_bbox.shape)

        ''' 
        this removes invalid boxes like boxes having low confidence or 
        which has size greater than image

        '''
        #print("\n\t pred_bbox=",len(pred_bbox))
        bboxes = postprocess_boxes(pred_bbox, image2.numpy(), YOLO_INPUT_SIZE,0.25)
        #print("\n\t befor nms bboxes =",bboxes.shape)


        bboxes = nms(bboxes, 0.1, method='nms')
        
        
        #print("\n\t after nms 1.bboxes =",len(bboxes))
        #print("\n\t after nms 2.bboxes =",bboxes)
    
        #draw_bbox() missing 1 required positional argument: 'bboxes
        '''
             draw_bbox(nm,counter,level, image, bboxes, CLASSES=YOLO_COCO_CLASSES, show_label=True, show_confidence=True,
              Text_colors=(255, 255, 0), rectangle_colors='', tracking=False)
        '''
        #print("\n\t bboxes len:",len(bboxes))
        resultImage = draw_bbox(nm,counter,i,255*image2.numpy(), bboxes, CLASSES=YOLO_COCO_CLASSES, rectangle_colors="")
        cv2.imwrite(f'./pred/{counter}.png',resultImage)
        #input("check")


    return global_steps.numpy(), optimizer.lr.numpy(), giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()

#mAP_model = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES) # create second model to measure mAP


In [ ]:
best_val_loss = 1000
for epoch in range(100):
    print("\n\t epoch:",epoch)
    for counter,(image_data, target) in enumerate(trainset):
        nm=str(np.random.randint(10000))+".png"
        nm=str(counter)+".png"
        #print("\n\t image_data=",image_data.shape)    
        #print("\n\t target:",target[0][0].shape)
        #print("\n\t target:",target[1][0].shape)
        #print("\n\t target:",target[2][0].shape)

        results = train_step(nm,counter,image_data, target)

        cur_step = results[0]%steps_per_epoch

        if 0:
            print("configure TEST options to validate model")
            yolo.save_weights(os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME))
            count, giou_val, conf_val, prob_val, total_val = 0., 0, 0, 0, 0
            
    print("epoch:{:2.0f} step:{:5.0f}/{}, lr:{:.6f}, giou_loss:{:7.2f}, conf_loss:{:7.2f}, prob_loss:{:7.2f}, total_loss:{:7.2f}"
          .format(epoch, cur_step, steps_per_epoch, results[1], results[2], results[3], results[4], results[5]))

 
    count, giou_val, conf_val, prob_val, total_val = 0., 0, 0, 0, 0


    try:
        print("\n\ngiou_val_loss:{:7.2f}, conf_val_loss:{:7.2f}, prob_val_loss:{:7.2f}, total_val_loss:{:7.2f}\n\n".
          format(giou_val/count, conf_val/count, prob_val/count, total_val/count))
    except Exception as e:
        pass
    
    if TRAIN_SAVE_CHECKPOINT and not TRAIN_SAVE_BEST_ONLY:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME+"_val_loss_{:7.2f}".format(total_val/count))
        yolo.save_weights(save_directory)
    if TRAIN_SAVE_BEST_ONLY and best_val_loss>total_val/(count+0.01):
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME)
        yolo.save_weights(save_directory)
        best_val_loss = total_val/(count+0.0001)
    if not TRAIN_SAVE_BEST_ONLY and not TRAIN_SAVE_CHECKPOINT:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME)
        yolo.save_weights(save_directory)




	 epoch: 0

	 respond_bbox: (1, 52, 52, 3, 1)

	 respond_bgd : (1, 52, 52, 3, 1)


best_val_loss = 1000
for epoch in range(100):
    print("\n\t epoch:",epoch)
    for counter,(image_data, target) in enumerate(trainset):
        nm=str(np.random.randint(10000))+".png"
        nm=str(counter)+".png"
        #print("\n\t image_data=",image_data.shape)    
        #print("\n\t target:",target[0][0].shape)
        #print("\n\t target:",target[1][0].shape)
        #print("\n\t target:",target[2][0].shape)

        results = train_step(nm,counter,image_data, target)

        cur_step = results[0]%steps_per_epoch
        print("epoch:{:2.0f} step:{:5.0f}/{}, lr:{:.6f}, giou_loss:{:7.2f}, conf_loss:{:7.2f}, prob_loss:{:7.2f}, total_loss:{:7.2f}"
              .format(epoch, cur_step, steps_per_epoch, results[1], results[2], results[3], results[4], results[5]))

        if 0:
            print("configure TEST options to validate model")
            yolo.save_weights(os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME))
            count, giou_val, conf_val, prob_val, total_val = 0., 0, 0, 0, 0
            
            '''
            for image_data, target in testset:
                results = validate_step(image_data, target)
                count += 1
                giou_val += results[0]
                conf_val += results[1]
                prob_val += results[2]
                total_val += results[3]
            # writing validate summary data
            with validate_writer.as_default():
                tf.summary.scalar("validate_loss/total_val", total_val/count, step=epoch)
                tf.summary.scalar("validate_loss/giou_val", giou_val/count, step=epoch)
                tf.summary.scalar("validate_loss/conf_val", conf_val/count, step=epoch)
                tf.summary.scalar("validate_loss/prob_val", prob_val/count, step=epoch)
            validate_writer.flush()

            print("\n\ngiou_val_loss:{:7.2f}, conf_val_loss:{:7.2f}, prob_val_loss:{:7.2f}, total_val_loss:{:7.2f}\n\n".
                  format(giou_val/count, conf_val/count, prob_val/count, total_val/count))

            '''

    count, giou_val, conf_val, prob_val, total_val = 0., 0, 0, 0, 0

    '''
    for image_data, target in testset:
        results = validate_step(image_data, target)
        count += 1
        giou_val += results[0]
        conf_val += results[1]
        prob_val += results[2]
        total_val += results[3]
    # writing validate summary data
    with validate_writer.as_default():
        tf.summary.scalar("validate_loss/total_val", total_val/count, step=epoch)
        tf.summary.scalar("validate_loss/giou_val", giou_val/count, step=epoch)
        tf.summary.scalar("validate_loss/conf_val", conf_val/count, step=epoch)
        tf.summary.scalar("validate_loss/prob_val", prob_val/count, step=epoch)
    validate_writer.flush()
    '''
    try:
        print("\n\ngiou_val_loss:{:7.2f}, conf_val_loss:{:7.2f}, prob_val_loss:{:7.2f}, total_val_loss:{:7.2f}\n\n".
          format(giou_val/count, conf_val/count, prob_val/count, total_val/count))
    except Exception as e:
        pass
    
    if TRAIN_SAVE_CHECKPOINT and not TRAIN_SAVE_BEST_ONLY:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME+"_val_loss_{:7.2f}".format(total_val/count))
        yolo.save_weights(save_directory)
    if TRAIN_SAVE_BEST_ONLY and best_val_loss>total_val/(count+0.01):
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME)
        yolo.save_weights(save_directory)
        best_val_loss = total_val/(count+0.0001)
    if not TRAIN_SAVE_BEST_ONLY and not TRAIN_SAVE_CHECKPOINT:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME)
        yolo.save_weights(save_directory)



In [ ]:
#! python train.py